# Example of XML caching for pydov

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fcaching.ipynb)

## Introduction

To speed up subsequent queries involving similar data, pydov uses a caching mechanism where raw DOV XML data is cached locally for later reuse. For regular usage of the package and data requests, the cache will be a *convenient* feature speeding up the time for subsequent queries. However, in case you want to alter the configuration or cache handling, this notebook illustrates some use cases on the cache handling.

### Use cases:
* Check cached files
* Speed up subsequent queries
* Disabling the cache
* Changing the location of cached data
* Changing the maximum age of cached data
* Cleaning the cache

In [1]:
# check pydov path
import warnings; warnings.simplefilter('ignore')
import pydov

## Use cases

### Check cached files

In [2]:
from pydov.search.boring import BoringSearch
boring = BoringSearch()

The `pydov.cache.cachedir` defines the directory on the file system used to cache DOV files:

In [3]:
# check the cache dir
import os
import pydov.util.caching
cachedir = pydov.cache.cachedir
print(cachedir)
print('directories: ', os.listdir(cachedir))

/tmp/pydov
directories:  ['filter', 'boring', 'sondering', 'grondmonster']


### Speed up subsequent queries

To illustrate the convenience of the caching during subsequent data requests, consider the following request, while measuring the time:

In [4]:
from pydov.util.location import Within, Box

# Get all borehole data in a bounding box (llx, llxy, ulx, uly) and timeit
%time df = boring.search(location=Within(Box(150145, 205030, 155150, 206935)))

[000/001] .
[000/255] ..................................................
[050/255] ..................................................
[100/255] ..................................................
[150/255] ..................................................
[200/255] ..................................................
[250/255] .....
CPU times: user 2.86 s, sys: 241 ms, total: 3.11 s
Wall time: 30.3 s


In [5]:
# The structure of cachedir implies a separate directory for each query type, since permalinks are not unique across types
# In this example 'boring' will be queried, therefore list xmls in the cache of the 'boring' type
# list files present
print('number of files: ', len(os.listdir(os.path.join(pydov.cache.cachedir, 'boring'))))
print('files present: ', os.listdir(os.path.join(pydov.cache.cachedir, 'boring')))

number of files:  736
files present:  ['2023-206524.xml.gz', '1986-059816.xml.gz', '1966-068248.xml.gz', '2021-196430.xml.gz', '1890-111945.xml.gz', '2023-203136.xml.gz', '2016-141571.xml.gz', '2019-166049.xml.gz', '2022-195501.xml.gz', '2018-170389.xml.gz', '1928-103238.xml.gz', '1882-112987.xml.gz', '2020-169408.xml.gz', '2021-181731.xml.gz', '2020-172695.xml.gz', '1986-005598.xml.gz', '1986-005594.xml.gz', '2019-168260.xml.gz', '2022-201616.xml.gz', '2022-191134.xml.gz', '2020-176580.xml.gz', '2019-168265.xml.gz', '2016-134350.xml.gz', '2022-191184.xml.gz', '1961-068369.xml.gz', '1970-061366.xml.gz', '1952-068355.xml.gz', '2023-203141.xml.gz', '2020-175375.xml.gz', '1936-087893.xml.gz', '2022-191158.xml.gz', '2016-134368.xml.gz', '2021-196427.xml.gz', '2020-175376.xml.gz', '1970-061365.xml.gz', '1988-091630.xml.gz', '1890-111943.xml.gz', '2022-191151.xml.gz', '1879-121412.xml.gz', '2023-203143.xml.gz', '2022-199618.xml.gz', '1970-061443.xml.gz', '1891-091514.xml.gz', '2023-202562.xm

Rerun the previous request and timeit again:

In [6]:
%time df = boring.search(location=Within(Box(150145, 205030, 155150, 206935)))

[000/001] .
[000/255] cccccccccccccccccccccccccccccccccccccccccccccccccc
[050/255] cccccccccccccccccccccccccccccccccccccccccccccccccc
[100/255] cccccccccccccccccccccccccccccccccccccccccccccccccc
[150/255] cccccccccccccccccccccccccccccccccccccccccccccccccc
[200/255] cccccccccccccccccccccccccccccccccccccccccccccccccc
[250/255] ccccc
CPU times: user 395 ms, sys: 81.3 ms, total: 476 ms
Wall time: 708 ms


The use of the cache decreased the runtime by a factor 100 in the current example. This will increase drastically if more permalinks are queried since the download takes much longer than the IO at runtime.

### Disabling the cache

You can (temporarily!) disable the caching mechanism. This disables both the saving of newly downloaded data in the cache, 
as well as reusing existing data in the cache. It remains valid for the time being of the instantiated pydov.cache object.
It does not delete existing data in the cache.

In [7]:
# list number of files
print('number of files: ', len(os.listdir(os.path.join(cachedir, 'boring'))))

number of files:  736


In [8]:
# disable caching
cache_orig = pydov.cache
pydov.cache = None
# new query
df = boring.search(location=Within(Box(151000, 205930, 153000, 206000)))
print(df.head())

[000/001] .
[000/002] ..
                                         pkey_boring     boornummer         x  \
0  https://www.dov.vlaanderen.be/data/boring/1895...   kb15d43w-B47  151600.0   
1  https://www.dov.vlaanderen.be/data/boring/1984...  kb15d43w-B403  151041.0   

          y  mv_mtaw  start_boring_mtaw   gemeente  diepte_boring_van  \
0  205998.0    15.00              15.00  Antwerpen                0.0   
1  205933.0    21.07              21.07  Antwerpen                0.0   

   diepte_boring_tot datum_aanvang                                uitvoerder  \
0                3.3    1895-01-04                                  onbekend   
1                7.0    1984-09-26  Universiteit Gent - Geologisch Instituut   

   boorgatmeting  diepte_methode_van  diepte_methode_tot   boormethode  
0          False                 0.0                 3.3      onbekend  
1          False                 0.0                 7.0  droge boring  


In [9]:
# list number of files
print('number of files: ', len(os.listdir(os.path.join(cachedir, 'boring'))))

number of files:  736


Hence, no new files were added to the cache when disabling it.

The caching is disabled by removing the pydov.cache object from the namespace. If you want to enable caching again you must instantiate it anew.

In [10]:
pydov.cache = cache_orig

### Changing the location of cached data

By default, pydov stores the cache in a temporary directory provided by the user's operating system. On Windows, the cache is usually located in:  `C:\Users\username\AppData\Local\Temp\pydov\`
If you want the cached xml files to be saved in another location you can define your own cache for the current runtime. Mind that this does not change the location of previously saved data. No lookup in the old datafolder will be performed after changing the directory's location.
Besides controlling the cache's location, this also allows using different scripts or projects.

In [11]:
import pydov.util.caching

pydov.cache = pydov.util.caching.GzipTextFileCache(
    cachedir=r'C:\temp\pydov'
    )

In [12]:
cachedir = pydov.cache.cachedir
print(cachedir)

C:\temp\pydov


In [13]:
# for the sake of the example, change dir location back 
pydov.cache = cache_orig
cachedir = pydov.cache.cachedir

### Changing the maximum age of cached data

If you work with rapidly changing data or want to control when cached data is renewed, you can do so by changing the maximum age of cached data to be considered valid for the currenct runtime. You can use 'weeks', 'days' or any other common datetime format.
If a cached version exists and is younger than the maximum age, it is used in favor of renewing the data from DOV services. If no cached version exists or is older than the maximum age, the data is renewed and saved in the cache.
Note that data older than the maximum age is not automatically deleted from the cache.

In [14]:
import pydov.util.caching
import datetime
pydov.cache = pydov.util.caching.GzipTextFileCache(
    max_age=datetime.timedelta(seconds=1)
    )
print(pydov.cache.max_age)

0:00:01


In [15]:
from time import ctime
print(os.listdir(os.path.join(cachedir, 'boring'))[0])
ctime(os.path.getmtime(os.path.join(os.path.join(cachedir, 'boring'),
                                    os.listdir(os.path.join(cachedir, 'boring'))[0]
                                   )
                      )
     )

2023-206524.xml.gz


'Tue Oct  3 14:28:24 2023'

In [16]:
# rerun previous query 
%time df = boring.search(location=Within(Box(150145, 205030, 155150, 206935)))

[000/001] .
[000/255] ..................................................
[050/255] ..................................................
[100/255] ..................................................
[150/255] ..................................................
[200/255] ..................................................
[250/255] .....
CPU times: user 2.54 s, sys: 348 ms, total: 2.89 s
Wall time: 35.5 s


In [17]:
from time import ctime
print(os.listdir(os.path.join(cachedir, 'boring'))[0])
ctime(os.path.getmtime(os.path.join(os.path.join(cachedir, 'boring'),
                                    os.listdir(os.path.join(cachedir, 'boring'))[0]
                                   )
                      )
     )

2023-206524.xml.gz


'Tue Oct  3 14:28:24 2023'

### Cleaning the cache

Since we use a temporary directory provided by the operating system, we rely on the operating system to clean the folder when it deems necessary.

To clean the cache, removing all records older than the maximum age

In [18]:
from time import sleep

In [19]:
print('number of files before clean: ', len(os.listdir(os.path.join(cachedir, 'boring'))))
sleep(2) # remember we've put the caching age on 1 second
pydov.cache.clean()
print('number of files after clean: ', len(os.listdir(os.path.join(cachedir, 'boring'))))

number of files before clean:  736
number of files after clean:  0


Should you want to remove the pydov cache from code yourself, you can do so as illustrated below. Note that this will erase the entire cache, not only the records older than the maximum age:

In [20]:
pydov.cache.remove()
# check existence of the cache directory:
print(os.path.exists(os.path.join(cachedir, 'boring')))

False


### Disabling stale responses on error

By default, pydov will return stale data (i.e. XML documents still present in
the cache, but no longer considered valid) in case it fails to download a fresh
copy from the DOV webservices. We believe this behaviour to benefit most users, as we think stale data is still better than no data at all.  

If your application cannot afford stale data, you can switch the default
behaviour by issuing:

In [21]:
pydov.cache.stale_on_error = False

This will cause pydov not to return stale data and instead set the XML fields
to NaN, as if the stale data wasn't available.

### Custom caching

By default, pydov caches files on disk as gzipped XML documents. Should you
for any reason want to use plain text XML documents instead, you can do so by
using the PlainTextFileCache instead of the GzipTextFileCache.  
Mind that this can increase the disk usage of the cache by 10x.:

In [22]:
import pydov.util.caching
pydov.cache = pydov.util.caching.PlainTextFileCache()

#### Implementing custom caching

Should you want to implement your own caching mechanism, you can do so by
subclassing :class:`pydov.util.caching.AbstractCache` and implementing its
abstract methods ``get``, ``clean`` and ``remove``. Hereby you can use the
available methods ``_get_remote`` to request data from the DOV webservices
and ``_emit_cache_hit`` to notify hooks a file has been retrieved from the
cache.

Note that the ``get`` method will be called from multiple threads
simultaneously, so implementations must be threadsafe or use locking.

A (naive) implementation for an in-memory cache would be something like:

In [23]:
from pydov.util.caching import AbstractCache

class MemoryCache(AbstractCache):
    def __init__(self):
        self.cache = {}

    def get(self, url):
        if url not in self.cache:
            self.cache[url] = self._get_remote(url)
        else:
            self._emit_cache_hit(url)
        return self.cache[url]

    def clean(self):
        self.cache = {}

    def remove(self):
        self.cache = {}

pydov.cache = MemoryCache()